In [1]:
from trainer import Trainer
from model.base_model_fine_tune import BaseNetTuned
import pathlib
import utils
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix
from mlxtend.plotting import plot_confusion_matrix

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

%matplotlib inline
%load_ext autoreload
%autoreload 2

So we have a problem - our `resnet50` just doesn't learn anything. How could this be? Let's try to use it with simple datasets: mnist and cats and dogs.

## model and data

In [5]:
def get_model():
    base_model = tf.keras.applications.ResNet50(input_shape=(224, 224, 3),
                                            include_top=False,
                                            weights='imagenet')
    base_model.trainable = False
    
    model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=2,
                          activation='softmax')
    ])
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    
    return model

In [4]:
def get_data(SPLIT_WEIGHTS = (8, 1, 1), 
             BATCH_SIZE = 32, 
             SHUFFLE_BUFFER_SIZE = 1000):
    
    # load raw data
    splits = tfds.Split.TRAIN.subsplit(weighted=SPLIT_WEIGHTS)
    (raw_train, raw_validation, raw_test), metadata = tfds.load(
        'cats_vs_dogs', 
        split=list(splits),
        with_info=True, 
        as_supervised=True)
    
    # preprocess data
    def format_example(image, label, IMG_SIZE = 224):
        image = tf.cast(image, tf.float32)
        image /= 255.0
        image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
        return image, label
    
    train = raw_train.map(format_example)
    validation = raw_validation.map(format_example)
    test = raw_test.map(format_example)

    # shuffle (only for training data) and batch
    train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
    validation_batches = validation.batch(BATCH_SIZE)
    test_batches = test.batch(BATCH_SIZE)
    
    return train_batches, validation_batches, test_batches

## initial training

In [6]:
# get model and data
model = get_model()
train_batches, validation_batches, test_batches = get_data()

In [9]:
def initial_training(BATCH_SIZE=32, SPLIT_WEIGHTS = (8, 1, 1)):
    
    # get training and validation steps
    num_train, num_val, num_test = (
      metadata.splits['train'].num_examples*weight/10
      for weight in SPLIT_WEIGHTS)
    
    initial_epochs = 10
    steps_per_epoch = round(num_train)//BATCH_SIZE
    validation_steps = 20
    
    history = model.fit(train_batches,
                    epochs=initial_epochs,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=validation_batches,
                    validation_steps=validation_steps
                    )
    
    return history

In [ ]:
history = initial_training()

In [ ]:
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(15, 5))
    plt.subplot(1, 2, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,1.0])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

## fine tuning